In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
weather_data=pd.read_csv(r"C:\Users\cynth\Desktop\Python API\Python-APIs\WeatherPy\Data\weather_data.csv")
weather_data.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Vaini,TO,1596815759,-21.20,-175.20,64.4,88,20,4.70
1,1,Pisco,PE,1596815760,-13.70,-76.22,63.0,82,75,3.36
2,2,Hambantota,LK,1596815760,6.12,81.12,82.4,83,40,17.22
3,3,Busselton,AU,1596815760,-33.65,115.33,52.0,55,100,15.14
4,4,Saint Paul Harbor,US,1596815760,57.79,-152.41,53.6,81,90,3.36


In [3]:
from config import g_key
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data[["Latitude", "Longitude"]].astype(float)


hum_rate = weather_data["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
#max temperature greater than 75 and less than 85
temp=weather_data.loc[weather_data['Max Temperature']>75,:]
temp_df=temp.loc[weather_data['Max Temperature']<85,:]
temp_df.head()
#wind speed greater than 10 mph
wind=temp_df.loc[temp_df['Wind Speed']>10,:]
wind.head()
#Humidity less than 75
humidity_df=wind.loc[wind['Humidity']<75,:]
humidity_df.head()


,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
134,134,Atuona,PF,1596815773,-9.80,-139.03,78.01,72,15,24.96
142,142,Port-Gentil,GA,1596815774,-0.72,8.78,82.40,65,20,12.48
169,169,Arraial do Cabo,BR,1596815776,-22.97,-42.03,80.60,64,40,18.34
184,184,Ribeira Grande,PT,1596815778,38.52,-28.70,77.00,65,75,12.75
260,260,Banda Aceh,ID,1596815785,5.56,95.32,81.95,71,100,11.56


In [28]:
hotel_df=pd.DataFrame()
total_df=pd.DataFrame(humidity_df)
hotel_df['City']=total_df['City']
hotel_df['Country']=total_df['Country']
hotel_df['Latitude']=total_df['Latitude']
hotel_df['Longitude']=total_df['Longitude']
hotel_df.head()

,City,Country,Latitude,Longitude
134,Atuona,PF,-9.80,-139.03
142,Port-Gentil,GA,-0.72,8.78
169,Arraial do Cabo,BR,-22.97,-42.03
184,Ribeira Grande,PT,38.52,-28.70
260,Banda Aceh,ID,5.56,95.32


In [30]:
for index, row in hotel_df.iterrows():

    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    country = row['Country']

    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    
    results = response['results']
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 134: Atuona.
Closest hotel to Atuona at -9.8 , -139.03 is Hôtel Hiva Oa Hanakee Lodge.
------------
Retrieving Results for Index 142: Port-Gentil.
Closest hotel to Port-Gentil at -0.72 , 8.78 is Iguane Café.
------------
Retrieving Results for Index 169: Arraial do Cabo.
Closest hotel to Arraial do Cabo at -22.97 , -42.03 is HI Marina dos Anjos.
------------
Retrieving Results for Index 184: Ribeira Grande.
Closest hotel to Ribeira Grande at 38.52 , -28.7 is Sixt Rent a Car.
------------
Retrieving Results for Index 260: Banda Aceh.
Closest hotel to Banda Aceh at 5.56 , 95.32 is Sei Hotel Banda Aceh.
------------
Retrieving Results for Index 269: Ozinki.
Closest hotel to Ozinki at 51.18 , 49.68 is Guz So "Ozinskaya Rb".
------------
Retrieving Results for Index 276: Touros.
Closest hotel to Touros at -5.2 , -35.46 is Correios, Agência Touros..
------------
Retrieving Results for Index 281: Mlandizi.
Closest hotel to Mlandizi at -6.7 , 38.73 is Mission to Un

In [31]:
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
134,Atuona,PF,-9.80,-139.03,Hôtel Hiva Oa Hanakee Lodge
142,Port-Gentil,GA,-0.72,8.78,Iguane Café
169,Arraial do Cabo,BR,-22.97,-42.03,HI Marina dos Anjos
184,Ribeira Grande,PT,38.52,-28.70,Sixt Rent a Car
260,Banda Aceh,ID,5.56,95.32,Sei Hotel Banda Aceh


In [35]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [38]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)
marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")
fig

ValueError: weights must be of the same length as locations or None